In [1]:
import os
import random
import shutil
import numpy as np

#=============================================================================================
# Precheck on directory structure
#=============================================================================================

if os.listdir('8-bit-c/test_data/cell/'):
    print 'Warning! Files detected in test data directory!'
    print 'Moving files back to training data directory...'

    for f in os.listdir('8-bit-c/test_data/cell/'):
        shutil.move('8-bit-c/test_data/cell/'+f,'8-bit-c/training_data/cell/'+f)

    for f in os.listdir('8-bit-c/test_data/nocell/'):
        shutil.move('8-bit-c/test_data/nocell/'+f,'8-bit-c/training_data/nocell/'+f)
        
    print 'Done!\n'
    
print "Splitting all training data into 70% training and 30% test data directories..."

cell_data = os.listdir('8-bit-c/training_data/cell/')
nocell_data = os.listdir('8-bit-c/training_data/nocell/')

validation_cell_data = random.sample(cell_data, int(0.3*len(cell_data)))
validation_nocell_data = random.sample(nocell_data, int(0.3*len(nocell_data)))

for f in validation_cell_data:
    shutil.move('8-bit-c/training_data/cell/'+f,'8-bit-c/test_data/cell/'+f)

for f in validation_nocell_data:
    shutil.move('8-bit-c/training_data/nocell/'+f,'8-bit-c/test_data/nocell/'+f)    

Warning! Files detected in test data directory!
Moving files back to training data directory...
Done!

Splitting all training data into 70% training and 30% test data directories...


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from hyperas import optim
from hyperopt import Trials, STATUS_OK, tpe
from hyperas.distributions import choice, uniform
import h5py
from keras.optimizers import RMSprop

# Define models to compare

def model(training_data, test_data):    
    img_width, img_height = 80, 80
    input_shape = (img_width, img_height, 1)
    
    epochs = 2
    batch_size = 32
    model = Sequential()
    model_choice = {{choice(['one', 'two', 'three'])}}
    if model_choice == 'one':
        model.add(Conv2D(64, (3, 3), input_shape = (80, 80, 1), activation='relu',padding='same'))
        model.add(Conv2D(64, (3, 3), input_shape = (80, 80, 1), activation='relu',padding='same'))
        model.add(AveragePooling2D(pool_size=2,strides=2))
        model.add(Conv2D(128, (3, 3), input_shape = (80, 80, 1), activation='relu'))
        model.add(Conv2D(128, (3, 3), input_shape = (80, 80, 1), activation='relu'))
        model.add(AveragePooling2D(pool_size=2,strides=2))
        model.add(Conv2D(256, (3, 3), input_shape = (80, 80, 1), activation='relu'))
        model.add(Conv2D(256, (3, 3), input_shape = (80, 80, 1), activation='relu'))
        model.add(Conv2D(256, (3, 3), input_shape = (80, 80, 1), activation='relu'))
        model.add(AveragePooling2D(pool_size=2,strides=2))
        model.add(Conv2D(512, (3, 3), input_shape = (80, 80, 1), activation='relu'))
        model.add(Conv2D(512, (3, 3), input_shape = (80, 80, 1), activation='relu'))
        model.add(Conv2D(512, (3, 3), input_shape = (80, 80, 1), activation='relu'))
        model.add(AveragePooling2D(pool_size=3,strides=3))
        model.add(Conv2D(512, (3, 3), input_shape = (80, 80, 1), activation='relu'))
        model.add(Conv2D(512, (3, 3), input_shape = (80, 80, 1), activation='relu'))
        model.add(Conv2D(512, (3, 3), input_shape = (80, 80, 1), activation='relu'))
        model.add(Dropout({{uniform(0, 1)}})) 
    elif model_choice == 'two':
        model.add(Conv2D(32, (3, 3), input_shape = (80, 80, 1), activation='relu',padding='same'))
        model.add(Conv2D(32, (3, 3), input_shape = (80, 80, 1), activation='relu',padding='same'))
        model.add(MaxPooling2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
        model.add(Conv2D(64, (3, 3), input_shape = (80, 80, 1), activation='relu'))
        model.add(Conv2D(64, (3, 3), input_shape = (80, 80, 1), activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
# Below is Gerald's original model
    elif model_choice == 'three':  
        model.add(Conv2D(8, (3, 3), input_shape = (80, 80, 1), activation = 'relu'))
        model.add(MaxPooling2D(pool_size = (2, 2)))
        model.add(Conv2D(16, (3, 3), input_shape = (80, 80, 1), activation = 'relu'))
        model.add(MaxPooling2D(pool_size = (2, 2)))
        model.add(BatchNormalization())
        model.add(Conv2D(32, (3, 3), input_shape = (80, 80, 1), activation = 'relu'))
        model.add(MaxPooling2D(pool_size = (2, 2)))
        model.add(BatchNormalization())
        model.add(Conv2D(32, (3, 3), input_shape = (80, 80, 1), activation = 'relu'))
        model.add(MaxPooling2D(pool_size = (2, 2)))
        model.add(BatchNormalization())
        model.add(Dropout({{uniform(0, 1)}}))
#         model.add(Flatten())
        model.add(Dense({{choice([128, 256, 512])}}))
        model.add(Activation({{choice(['relu', 'sigmoid'])}}))
        model.add(Dropout({{uniform(0, 1)}}))
    
    model.add(Flatten())
    model.add(Dense(units = 1, activation = 'sigmoid')) #softmax
    model.compile(optimizer={{choice(['rmsprop', 'adam', 'sgd'])}}, loss = 'binary_crossentropy', metrics = ['accuracy'])

    model.summary()
    
    # Find number of epoch and validation steps
    steps_epoch = len([filename for filename in os.listdir('8-bit-c/training_data/cell') if filename.endswith(".tif")]) + len([filename for filename in os.listdir('8-bit-c/training_data/nocell') if filename.endswith(".tif")])
    steps_valid = len([filename for filename in os.listdir('8-bit-c/test_data/cell') if filename.endswith(".tif")]) + len([filename for filename in os.listdir('8-bit-c/test_data/nocell') if filename.endswith(".tif")])
    
    history = model.fit_generator(
        training_data,
        steps_per_epoch=steps_epoch,
        epochs=epochs,
        validation_data=test_data,
        validation_steps=steps_valid,
        verbose=0)

#     score, acc = model.evaluate_generator(generator=validation_generator, 
#                                       steps=nb_validation_samples // batch_size)

    validation_acc = np.amax(history.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)

    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

def data():

    # training data
    train_datagen = ImageDataGenerator(rotation_range=180, rescale = 1./255, shear_range = 0.15, zoom_range = 0.15, width_shift_range=0.15, height_shift_range=0.15, horizontal_flip = True, vertical_flip = True)
    training_data = train_datagen.flow_from_directory('8-bit-c/training_data', target_size = (80, 80), batch_size = 32, class_mode = 'binary', color_mode = 'grayscale')
    #training_data = train_datagen.flow_from_directory('8-bit-c/training_data', target_size = (80, 80), batch_size = 32, class_mode = 'binary', color_mode = 'grayscale', save_to_dir='preview', save_prefix='cell', save_format='jpeg')
    # validation data
    test_datagen = ImageDataGenerator(rescale = 1./255)
    test_data = test_datagen.flow_from_directory('8-bit-c/test_data', target_size = (80, 80), batch_size = 32, class_mode = 'binary', color_mode = 'grayscale')

    print "Done!\n"

    return training_data, test_data

if __name__ == '__main__':

    train_generator, validation_generator = data()

    best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          trials=Trials(),
                                          notebook_name='max_cc_hyperas_v4')


    print("Evalutation of best performing model:")
    print(best_model.evaluate_generator(validation_generator))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)

/opt/anaconda/envs/deeplearn/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 9800 images belonging to 2 classes.
Found 4202 images belonging to 2 classes.
Done!

>>> Imports:
#coding=utf-8

try:
    import os
except:
    pass

try:
    import random
except:
    pass

try:
    import shutil
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from keras.preprocessing.image import ImageDataGenerator
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Conv2D, MaxPooling2D
except:
    pass

try:
    from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ModelCheckpoint
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import h5py
except:
    pass

try:
    from keras

_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 40, 40, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 40, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 38, 38, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 36, 36, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 36, 36, 64)        256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 18, 18, 64)        0         
_________________________________________________________________
flatten_1 

_________________________________________________________________              
('Best validation acc of epoch:', 0.8857394866025484)                          
_________________________________________________________________              
Layer (type)                 Output Shape              Param #                 
conv2d_13 (Conv2D)           (None, 80, 80, 32)        320                     
_________________________________________________________________              
conv2d_14 (Conv2D)           (None, 80, 80, 32)        9248                    
_________________________________________________________________              
max_pooling2d_11 (MaxPooling (None, 40, 40, 32)        0                       
_________________________________________________________________              
dropout_7 (Dropout)          (None, 40, 40, 32)        0                       
_________________________________________________________________              
conv2d_15 (Conv2D)           (None, 38, 